# Agent's Performance - Deep Deterministic Policy Gradients (DDPG)
---
Notebook, addapted from 
https://raw.githubusercontent.com/udacity/deep-reinforcement-learning/master/ddpg-bipedal/DDPG.ipynb
training DDPG with OpenAI Gym's BipedalWalker-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from DDPG_agent import Agent

### 2. Instantiate the Environment and Agent

In [2]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
env = UnityEnvironment(file_name='C:\EigeneLokaleDaten\DeepRL\Value-based-methods\p2_continuous-control\Reacher_Windows_x86_64_single_agent\Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
#print('The state for the 2nd agent looks like:', states[1])

print(state_size)
print(len(states))

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  7.90150642e+00 -1.00000000e+00
  1.25147498e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -2.99753308e-01]
33
1


In [ ]:
#agent = Agent(state_size=env_info.vector_observations.shape[1], action_size=brain.vector_action_space_size, random_seed=10, NN_architecture=NN_architecture)

# Load the Network Weights and run Agent

In [16]:
data = np.load('state_scale.npz')
scale = data['scale_int']
rnd_seed = 42

NN_architecture = None # default 256,256,256,128
Noise_params = None # default noise
agent = Agent(state_size=env_info.vector_observations.shape[1], action_size=brain.vector_action_space_size, random_seed=rnd_seed, NN_architecture=NN_architecture, Noise_params=Noise_params, ReLU=True)
agent.critic_local.load_state_dict(torch.load('checkpoint_critic_250.pth'))
agent.critic_target.load_state_dict(torch.load('checkpoint_critic_250.pth'))
agent.actor_local.load_state_dict(torch.load('checkpoint_actor_250.pth'))
agent.actor_target.load_state_dict(torch.load('checkpoint_actor_250.pth'))

for runs in range(1):
    env_info = env.reset(train_mode=False)[brain_name]      # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    states = states/scale
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        #actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        actions = agent.act(states,add_noise=False) 
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states/scale                               # roll over states to next time step
        dones = env_info.local_done                        # see if episode finished
        #states_mean.append(states[0])
        if np.any(dones):                                  # exit loop if episode finished
            break        
    print(runs,'Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))


Hyperparams 1000000 64 0.99 0.001 0.0001 0.001 0 1
Hyperparams Noise 0.0 0.15 0.2
0 Total score (averaged over agents) this episode: 32.629999270662665


### 5. Explore

In this exercise, we have provided a sample DDPG agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster than this benchmark implementation.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task!
- Write your own DDPG implementation.  Use this code as reference only when needed -- try as much as you can to write your own algorithm from scratch.
- You may also like to implement prioritized experience replay, to see if it speeds learning.  
- The current implementation adds Ornsetein-Uhlenbeck noise to the action space.  However, it has [been shown](https://blog.openai.com/better-exploration-with-parameter-noise/) that adding noise to the parameters of the neural network policy can improve performance.  Make this change to the code, to verify it for yourself!
- Write a blog post explaining the intuition behind the DDPG algorithm and demonstrating how to use it to solve an RL environment of your choosing.  